In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Daten laden
# Wir laden die vorbereiteten Daten aus einer CSV-Datei. Die Datei sollte vorher aufbereitet worden sein.
data = pd.read_csv('processed_sentiment140.csv')  # Passe den Pfad an die Datei an.

# NaN-Werte entfernen oder ersetzen
# Leere oder fehlende Texte werden durch einen leeren String ersetzt, damit keine Fehler entstehen.
data['processed_text'] = data['processed_text'].fillna('')

# Aufteilen in Merkmale (X) und Ziel (y)
# 'processed_text' ist die vorbereitete Textspalte, 'Sentiment' das Ziel (positiv/negativ).
X = data['processed_text']
y = data['Sentiment']

# Daten in Trainings- und Testdaten aufteilen
# Wir teilen die Daten in Trainings- und Testdatensätze auf, um das Modell zu trainieren und zu evaluieren.
# Stratify sorgt dafür, dass die Verteilung der Klassen im Training und Test gleich ist.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# TF-IDF-Transformation
# Der Text wird in numerische Merkmale umgewandelt. 
# Wir verwenden TF-IDF (Term Frequency-Inverse Document Frequency), um die Bedeutung von Wörtern zu gewichten.
tfidf_vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)  # Anpassen und Transformieren des Trainingsdatensatzes
X_test_tfidf = tfidf_vectorizer.transform(X_test)  # Transformieren des Testdatensatzes

# Fortschrittsanzeige für das Training
# Wir verwenden `tqdm`, um den Fortschritt während des Modelltrainings anzuzeigen.
print("Training des Modells...")
for _ in tqdm(range(1), desc="Training Fortschritt"):
    # Logistische Regression
    # Wir verwenden Logistic Regression mit einem sparsamen Solver ('saga').
    # Der Solver unterstützt größere Daten und L1/L2-Regularisierung.
    logistic_model = LogisticRegression(max_iter=1000, solver='saga', verbose=1)
    logistic_model.fit(X_train_tfidf, y_train)  # Das Modell wird auf den Trainingsdaten trainiert.

# Vorhersage und Evaluierung
# Das trainierte Modell wird verwendet, um Vorhersagen für den Testdatensatz zu treffen.
print("\nEvaluierung des Modells...")
y_pred = logistic_model.predict(X_test_tfidf)

# Ergebnisse ausgeben
# Wir berechnen die Genauigkeit (Accuracy) und geben einen detaillierten Klassifikationsbericht aus.
accuracy = accuracy_score(y_test, y_pred)
classification_report_text = classification_report(y_test, y_pred)

print(f"\nAccuracy: {accuracy}")
print(f"Classification Report:\n{classification_report_text}")

# Ergebnisse zusammenführen
# Wir fügen den ursprünglichen Text, die wahre Klasse und die vorhergesagte Klasse in einem DataFrame zusammen.
results = pd.DataFrame({
    'text': X_test,
    'true_label': y_test,
    'predicted_label': y_pred
})

# Beispiel korrekt klassifizierter Einträge
# Zufällige Auswahl von 5 korrekt klassifizierten Beispielen.
correct_predictions = results[results['true_label'] == results['predicted_label']].sample(5, random_state=42)

# Beispiel falsch klassifizierter Einträge
# Zufällige Auswahl von 5 falsch klassifizierten Beispielen.
incorrect_predictions = results[results['true_label'] != results['predicted_label']].sample(5, random_state=42)

# Anzeigen der Beispiele
# Beispiele für die Analyse des Modells ausgeben.
print("\nKorrekt klassifizierte Beispiele:")
print(correct_predictions)

print("\nFalsch klassifizierte Beispiele:")
print(incorrect_predictions)


Training des Modells...


Training Fortschritt:   0%|                               | 0/1 [00:00<?, ?it/s][Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.1s finished
Training Fortschritt: 100%|███████████████████████| 1/1 [00:07<00:00,  7.19s/it]

convergence after 23 epochs took 7 seconds

Evaluierung des Modells...



Accuracy: 0.7647625
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.74      0.76    160000
           1       0.75      0.79      0.77    160000

    accuracy                           0.76    320000
   macro avg       0.77      0.76      0.76    320000
weighted avg       0.77      0.76      0.76    320000


Korrekt klassifizierte Beispiele:
                                                      text  true_label  \
78261    dont feel good think need sleep oh thanks hanging           0   
593236   hate smartarse rooky police power trip pull is...           0   
1552465  james play together least old time love busted...           1   
894162    rampr week going awesome im sure great good luck           1   
258270               sad win emerald samuelgordon congrats           0   

         predicted_label  
78261                  0  
593236                 0  
1552465                1  
894162                 1  
258270        